In [17]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

## Functions

In [9]:
# functions
def num_age(x):
    '''make age categories numeric'''
    if x == '18-39':
        return 0
    elif x == '40-44':
        return 1
    elif x == '45-49':
        return 2
    elif x == '50-54':
        return 3
    elif x == '55-59':
        return 4
    elif x == '60-64':
        return 5
    elif x == '65-69':
        return 6
    return 7


def cat_clean(x):
    '''binary columns were floats, make them ints'''
    return int(x)


def prep(df):
    '''prepare dataframe for EDA'''
    df['age'] = df['age'].apply(lambda x: num_age(x))
    df['clear'] = df['clear'].apply(lambda x: cat_clean(x))
    df['overcast'] = df['overcast'].apply(lambda x: cat_clean(x))
    df['partially_cloudy'] = df['partially_cloudy'].apply(lambda x: cat_clean(x))
    df['rain'] = df['rain'].apply(lambda x: cat_clean(x))

    cols = ['year', 'age', 'male', 'time_seconds', 'minimum_temperature',
           'maximum_temperature', 'temperature', 'relative_humidity', 'wind_speed',
           'precipitation', 'precipitation_cover', 'cloud_cover', 'clear',
           'overcast', 'partially_cloudy', 'rain']

    for col in cols:
        df[col] = pd.to_numeric(df[col], downcast='unsigned')

    df.drop(columns=['maximum_temperature', 'minimum_temperature'], inplace=True)

    event_dict = {year: df[df['year'] == year] for year in df['year'].unique()}
    
    return event_dict


def usable(event_dict, num):
    '''specify which years are usable based on minimum number of participants
    returns dict with years that exceed minimum values are correspoding df'''
    can_use = []
    for year in event_dict.keys():
        if len(event_dict[year]) > num:
            can_use.append(year)
    return {year: event_dict[year] for year in can_use}


def top_n(event_dict, num):
    '''take in event dict and first num of finishers that are desired,
    return df of top n racers for all years, use in conjuction with usable()'''
    df = pd.DataFrame()
    for year in event_dict.keys():
        df = pd.concat([df, event_dict[year].sort_values(by='time_seconds').iloc[:num]], ignore_index=True)
    return df
        
    
def n_range(event_dict, low, high):
    '''take in dictionary of dfs from prep(), upper percentage as float, lower percentage as float
    return df'''
    df = pd.DataFrame()
    for year in event_dict.keys():
        df = pd.concat([df, event_dict[year].sort_values(by='time_seconds').iloc[int(len(event_dict[year]) * low): int(len(event_dict[year]) * high)]], ignore_index=True)
    return df

def model_scores(X_test, y_test, model):
    nl = "\n"
    y_preds = model.predict(X_test)
    return print(f'R2: {model.score(X_test, y_test)}{nl}MAE: {mean_absolute_error(y_test, y_preds)}{nl}MSE: {mean_squared_error(y_test, y_preds)}{nl}RMSE: {mean_squared_error(y_test, y_preds, squared=False)}')

def undo_age(x):
    '''make age categories their division again'''
    if x == 0.0:
        return '18-39'
    elif x == 1.0:
        return '40-44'
    elif x == 2.0:
        return '45-49'
    elif x == 3.0:
        return '50-54'
    elif x == 4.0:
        return '55-59'
    elif x == 5.0:
        return '60-64'
    elif x == 6.0:
        return '65-69'
    return '70+'

## model

In [10]:
london = pd.read_csv('./data/London_Data/Clean/Clean_London_Results_Weather.csv')
nyc = pd.read_csv('./data/NYC_Data/Clean/Clean_NYC_Results_Weather.csv')
boston = pd.read_csv('./data/Boston_Data/Clean/Clean_Boston_Results_Weather.csv')
berlin = pd.read_csv('./data/Berlin_Data/Clean/Clean_Berlin_Results_Weather.csv')
chicago = pd.read_csv('./data/Chicago_Data/Clean/Clean_Chicago_Results_Weather.csv')

london_dict = prep(london)
nyc_dict = prep(nyc)
boston_dict = prep(boston)
berlin_dict = prep(berlin)
chicago_dict = prep(chicago)

london = n_range(usable(london_dict, 9_900), 0, 1)
nyc = n_range(usable(nyc_dict, 9_900), 0, 1)
boston = n_range(usable(boston_dict, 9_900), 0, 1)
berlin = n_range(usable(berlin_dict, 9_900), 0, 1)
chicago = n_range(usable(chicago_dict, 9_900), 0, 1)

aggregate = {col: 'mean' for col in london.drop(columns=['year'])}

london = london.groupby(['year', 'age', 'male']).agg(aggregate)
nyc = nyc.groupby(['year', 'age', 'male']).agg(aggregate)
boston = boston.groupby(['year', 'age', 'male']).agg(aggregate)
berlin = berlin.groupby(['year', 'age', 'male']).agg(aggregate)
chicago = chicago.groupby(['year', 'age', 'male']).agg(aggregate)

events = [boston, berlin, chicago, london, nyc]
count = 0
for event in events:
    event['event'] = count
    count += 1
    
combined = pd.concat(events)
combined = combined.merge(pd.get_dummies(combined['event'], drop_first=True), left_index=True, right_index=True).drop(columns='event')
combined.drop(columns=['clear', 'overcast', 'partially_cloudy', 'rain'], inplace=True)
combined.columns = ['age', 'male', 'time_seconds', 'temperature', 'relative_humidity', 'wind_speed', 'precipitation', 'precipitation_cover', 'cloud_cover', 'boston', 'chicago', 'london', 'nyc']


berlin['age'] = berlin['age'].apply(lambda x: undo_age(x))
berlin = berlin.merge(pd.get_dummies(berlin['age'], drop_first=True), left_index=True, right_index=True).drop(columns='age')
berlin.drop(columns=['clear', 'overcast', 'partially_cloudy', 'rain'], inplace=True)

boston['age'] = boston['age'].apply(lambda x: undo_age(x))
boston = boston.merge(pd.get_dummies(boston['age'], drop_first=True), left_index=True, right_index=True).drop(columns='age')
boston.drop(columns=['clear', 'overcast', 'partially_cloudy', 'rain'], inplace=True)

chicago['age'] = chicago['age'].apply(lambda x: undo_age(x))
chicago = chicago.merge(pd.get_dummies(chicago['age'], drop_first=True), left_index=True, right_index=True).drop(columns='age')
chicago.drop(columns=['clear', 'overcast', 'partially_cloudy', 'rain'], inplace=True)

london['age'] = london['age'].apply(lambda x: undo_age(x))
london = london.merge(pd.get_dummies(london['age'], drop_first=True), left_index=True, right_index=True).drop(columns='age')
london.drop(columns=['clear', 'overcast', 'partially_cloudy', 'rain'], inplace=True)

nyc['age'] = nyc['age'].apply(lambda x: undo_age(x))
nyc = nyc.merge(pd.get_dummies(nyc['age'], drop_first=True), left_index=True, right_index=True).drop(columns='age')
nyc.drop(columns=['clear', 'overcast', 'partially_cloudy', 'rain'], inplace=True)

combined['age'] = combined['age'].apply(lambda x: undo_age(x))
combined = combined.merge(pd.get_dummies(combined['age'], drop_first=True), left_index=True, right_index=True).drop(columns='age')

In [60]:
boost = XGBRegressor()

xparams = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07, .09, .11], #so called `eta` value
              'max_depth': [3, 4, 5, 6],
              'min_child_weight': [3, 4, 5],
              'subsample': [.5, .7, 1],
              'colsample_bytree': [.5, .7, 1],
              'n_estimators': [100, 200, 300, 400, 500]}

xgs = GridSearchCV(
    boost,
    xparams,
    cv=2,
    verbose=True
)

In [61]:
train = berlin[berlin.index.isin(range(2016), level=0)]
test = berlin[berlin.index.isin(range(2016,2019), level=0)]

X_train = train.drop(columns = ['time_seconds'])
X_test = test.drop(columns = ['time_seconds'])

y_train = train['time_seconds']
y_test = test['time_seconds']


xgs.fit(X_train, y_train)
model_scores(X_test, y_test, xgs)

Fitting 2 folds for each of 2700 candidates, totalling 5400 fits
R2: 0.6375618863304717
MAE: 826.415306901741
MSE: 861908.3651617399
RMSE: 928.390200918633


In [62]:
xgs.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.11,
 'max_depth': 5,
 'min_child_weight': 5,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

In [63]:
train = boston[boston.index.isin(range(2016), level=0)]
test = boston[boston.index.isin(range(2016,2019), level=0)]

X_train = train.drop(columns = ['time_seconds'])
X_test = test.drop(columns = ['time_seconds'])

y_train = train['time_seconds']
y_test = test['time_seconds']


xgs.fit(X_train, y_train)
model_scores(X_test, y_test, xgs)

Fitting 2 folds for each of 2700 candidates, totalling 5400 fits
R2: 0.5209362487824027
MAE: 1087.6172792968043
MSE: 1611207.4497266568
RMSE: 1269.3334667165507


In [64]:
xgs.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.07,
 'max_depth': 4,
 'min_child_weight': 3,
 'n_estimators': 400,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

In [65]:
train = chicago[chicago.index.isin(range(2016), level=0)]
test = chicago[chicago.index.isin(range(2016,2019), level=0)]

X_train = train.drop(columns = ['time_seconds'])
X_test = test.drop(columns = ['time_seconds'])

y_train = train['time_seconds']
y_test = test['time_seconds']

xgs.fit(X_train, y_train)
model_scores(X_test, y_test, xgs)

Fitting 2 folds for each of 2700 candidates, totalling 5400 fits
R2: 0.8406198101599024
MAE: 476.2787797007107
MSE: 418544.45662413136
RMSE: 646.9501191159419


In [66]:
xgs.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.11,
 'max_depth': 5,
 'min_child_weight': 5,
 'n_estimators': 200,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

In [67]:
train = london[london.index.isin(range(2016), level=0)]
test = london[london.index.isin(range(2016,2019), level=0)]

X_train = train.drop(columns = ['time_seconds'])
X_test = test.drop(columns = ['time_seconds'])

y_train = train['time_seconds']
y_test = test['time_seconds']

xgs.fit(X_train, y_train)
model_scores(X_test, y_test, xgs)

Fitting 2 folds for each of 2700 candidates, totalling 5400 fits
R2: 0.8887809405720463
MAE: 409.8236576324586
MSE: 272927.0003438948
RMSE: 522.424157504125


In [68]:
xgs.best_params_

{'colsample_bytree': 1,
 'learning_rate': 0.09,
 'max_depth': 6,
 'min_child_weight': 5,
 'n_estimators': 100,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

In [69]:
train = nyc[nyc.index.isin(range(2016), level=0)]
test = nyc[nyc.index.isin(range(2016,2019), level=0)]

X_train = train.drop(columns = ['time_seconds'])
X_test = test.drop(columns = ['time_seconds'])

y_train = train['time_seconds']
y_test = test['time_seconds']

xgs.fit(X_train, y_train)
model_scores(X_test, y_test, xgs)

Fitting 2 folds for each of 2700 candidates, totalling 5400 fits
R2: 0.9621004339577592
MAE: 304.40099237401137
MSE: 146607.58000598318
RMSE: 382.8936928260678


In [70]:
xgs.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.05,
 'max_depth': 5,
 'min_child_weight': 5,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

In [39]:
train = combined[combined.index.isin(range(2016), level=0)]
test = combined[combined.index.isin(range(2016,2019), level=0)]

X_train = train.drop(columns = ['time_seconds'])
X_test = test.drop(columns = ['time_seconds'])

y_train = train['time_seconds']
y_test = test['time_seconds']

xgs.fit(X_train, y_train)
model_scores(X_test, y_test, xgs)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
R2: 0.6720293793104591
MAE: 897.8322815671098
MSE: 1195364.9227573343
RMSE: 1093.3274544972035


In [51]:
xgs.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.07,
 'max_depth': 3,
 'min_child_weight': 5,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.7}